In [4]:
from pyprojroot.here import here

here()
import pandas as pd
import duckdb
from janitor import clean_names
from pyarrow import parquet as pq
import duckdb
df = clean_names(pq.read_table(here(r"data/AllDaysEventos.parquet")).to_pandas())
df

,data_do_incidente,concelho,distrito,nivel_de_tensao,numero_de_eventos,dia,mes,ano,densidade_populacional,populacao_residente,...,indice_de_envelhecimento,rural,t_med,t_max,t_min,dd_med,dd_ffx,ff_med,ff_max,pr_qtd
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8,174592.0,...,142.4,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0,15188.0,...,219.9,1.000000,10.6,12.2,8.1,222.0,248.0,4.0,12.6,13.8
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1,57074.0,...,90.1,0.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2,80291.0,...,138.6,0.923077,14.4,15.2,12.0,232.0,236.0,5.4,13.4,3.8
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0,6929.0,...,419.1,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3,7993.0,...,163.9,1.000000,8.9,10.6,7.8,293.0,328.0,2.6,8.1,1.6
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1,15420.0,...,188.0,1.000000,NaN,NaN,NaN,337.0,339.0,2.6,10.1,5.8
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7,6042.0,...,398.0,1.000000,6.9,9.9,4.9,264.0,208.0,2.9,12.7,0.8
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6,4143.0,...,573.7,1.000000,6.9,9.9,4.9,264.0,208.0,2.9,12.7,0.8


In [14]:
duckdb.query("""
  select distinct nivel_de_tensao from df
  where numero_de_eventos > 1 
  and data_do_incidente >= '2018-01-01'
""").df()

,nivel_de_tensao
0,AT
1,MT
2,BT


In [29]:
# ver se há repetidos
duckdb.query("""
  select data_do_incidente, concelho, string_agg(nivel_de_tensao, ',') as niveis, count(*) as cnt from df
  where data_do_incidente >= '2018-01-01'
  group by data_do_incidente, concelho
  having cnt > 1
""").df()

,data_do_incidente,concelho,niveis,cnt
0,2021-05-31,Loures,"BT,MT",2
1,2018-06-21,Cartaxo,"BT,MT",2
2,2023-03-04,Almeirim,"AT - Alta tensão,MT",2
3,2021-08-13,Vila Real,"BT,MT",2
4,2019-05-09,Leiria,"BT,MT",2
5,2022-07-13,Setúbal,"AT,MT",2
6,2022-09-28,Lisboa,"BT,MT",2
7,2018-05-14,Alijó,"AT,MT",2
